In [1]:
Pkg.add("MXNet")

INFO: Package MXNet is already installed
INFO: METADATA is out-of-date — you may not have the latest version of MXNet
INFO: Use `Pkg.update()` to get the latest versions of your packages


In [7]:
using MXNet

mlp = @mx.chain mx.Variable(:data)             =>
  mx.FullyConnected(name=:fc1, num_hidden=128) =>
  mx.Activation(name=:relu1, act_type=:relu)   =>
  mx.FullyConnected(name=:fc2, num_hidden=64)  =>
  mx.Activation(name=:relu2, act_type=:relu)   =>
  mx.FullyConnected(name=:fc3, num_hidden=10)  =>
  mx.SoftmaxOutput(name=:softmax)

# data provider
batch_size = 100
include(Pkg.dir("MXNet", "examples", "mnist", "mnist-data.jl"))
train_provider, eval_provider = get_mnist_providers(batch_size)

# setup model
model = mx.FeedForward(mlp, context=mx.cpu())

# optimization algorithm
# where η is learning rate and μ is momentum
optimizer = MXNet.mx.SGD(lr=0.1, momentum=0.9) # η=0.1, μ=0.9

# fit parameters
mx.fit(model, optimizer, train_provider, n_epoch=20, eval_data=eval_provider)


INFO: Start training on MXNet.mx.Context[CPU0]
INFO: Initializing parameters...
INFO: Creating KVStore...
INFO: TempSpace: Total 0 MB allocated on CPU0
INFO: Start training...
INFO: == Epoch 001/020 ==========
INFO: ## Training summary
INFO:           accuracy = 0.7579
INFO:               time = 6.8185 seconds
INFO: ## Validation summary
INFO:           accuracy = 0.9539
INFO: == Epoch 002/020 ==========
INFO: ## Training summary
INFO:           accuracy = 0.9581
INFO:               time = 5.5357 seconds
INFO: ## Validation summary
INFO:           accuracy = 0.9635
INFO: == Epoch 003/020 ==========
INFO: ## Training summary
INFO:           accuracy = 0.9703
INFO:               time = 5.7871 seconds
INFO: ## Validation summary
INFO:           accuracy = 0.9691
INFO: == Epoch 004/020 ==========
INFO: ## Training summary
INFO:           accuracy = 0.9766
INFO:               time = 5.6506 seconds
INFO: ## Validation summary
INFO:           accuracy = 0.9697
INFO: == Epoch 005/020 =========

In [8]:
probs = mx.predict(model, eval_provider)

# collect all labels from eval data
labels = reduce(
  vcat,
  copy(mx.get(eval_provider, batch, :softmax_label)) for batch ∈ eval_provider)
# labels are 0...9
labels .= labels .+ 1

# Now we use compute the accuracy
pred = map(i -> indmax(probs[1:10, i]), 1:size(probs, 2))
correct = sum(pred .== labels)
accuracy = 100correct/length(labels)
@printf "Accuracy on eval set: %.2f%%\n" accuracy


INFO: TempSpace: Total 0 MB allocated on CPU0


Accuracy on eval set: 97.60%
